In [1]:
import json
import re
from typing import Optional

import pandas as pd

In [2]:
df = pd.concat([
    pd.read_csv('./data/llm_perf/hf-dgx-01_perf-report.csv'),
    pd.read_csv('./data/llm_perf/audace_perf-report.csv')
])

In [3]:
df['gpu'] = df['environment.gpus'].apply(lambda x: json.loads(x.replace('\'', '"'))[0])

In [4]:
def parse_model_parameters_count(model_name: str) -> Optional[tuple[float, str]]:
    match = re.search(r'([0-9.]+)(b|m)', model_name, re.IGNORECASE)
    if match is not None:
        count, unit = float(match[1]), match[2].lower()
        if unit == 'm':
            count /= 1000
        return count

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1944 entries, 0 to 175
Columns: 102 entries, forward.latency(s) to gpu
dtypes: bool(19), float64(27), int64(25), object(31)
memory usage: 1.3+ MB


In [6]:
df['parameters_count'] = df['model'].apply(parse_model_parameters_count)

In [7]:
df['energy_per_token'] = df['generate.energy_consumption(kWh/token)'] * 1000

In [8]:
df.rename(columns={'generate.throughput(tokens/s)':'throughput', 'generate.latency(s)':'latency'}, inplace=True)

In [9]:
sub = df[[
    'model',
    'parameters_count',
    'gpu',
    'energy_per_token',
    'throughput',
    'latency',
    'experiment_name'
]].dropna()



In [10]:
len(sub)

1692

In [11]:
sub.to_csv('./data/aggregated_llm_perf.csv', index=False)